---

In [1]:
"""
============================================================================
ЯЧЕЙКА 1: ИМПОРТЫ И НАСТРОЙКИ
============================================================================
Тестирование OCR моделей с параллельной обработкой через ThreadPoolExecutor
"""

# ============================================================================
# СТАНДАРТНЫЕ БИБЛИОТЕКИ
# ============================================================================
import os
import sys
import json
import base64
import logging
import time
import csv
from pathlib import Path
from io import BytesIO
from datetime import datetime
from typing import List, Dict, Optional, Tuple, Any
from dataclasses import dataclass, field, asdict

# ============================================================================
# ПАРАЛЛЕЛЬНАЯ ОБРАБОТКА
# ============================================================================
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

# ============================================================================
# ВНЕШНИЕ БИБЛИОТЕКИ
# ============================================================================
from PIL import Image
import requests
from tqdm import tqdm
from openai import OpenAI

# ============================================================================
# ВИЗУАЛИЗАЦИЯ И АНАЛИЗ
# ============================================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ============================================================================
# НАСТРОЙКА ЛОГИРОВАНИЯ
# ============================================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('ocr_processing.log', encoding='utf-8')
    ]
)
logger = logging.getLogger(__name__)

# ============================================================================
# ГЛОБАЛЬНЫЕ ПЕРЕМЕННЫЕ
# ============================================================================
# Добавить корень проекта в PYTHONPATH
PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name != "Team_5_AI_Tutor":
    PROJECT_ROOT = PROJECT_ROOT.parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / "data"
IMAGES_DIR = PROJECT_ROOT / "notebooks" / "vlm-ingestion-pipeline" / "test_images"
OUTPUT_DIR = DATA_DIR / "test_results"

# Создаём директории если не существуют
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


print("✅ Ячейка 1: Импорты загружены")
print(f"   📂 Корневая директория: {PROJECT_ROOT}")
print(f"   📂 Директория с изображениями: {IMAGES_DIR}")
print(f"   📂 Директория для результатов: {OUTPUT_DIR}")


✅ Ячейка 1: Импорты загружены
   📂 Корневая директория: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor
   📂 Директория с изображениями: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/notebooks/vlm-ingestion-pipeline/test_images
   📂 Директория для результатов: /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/data/test_results


In [3]:
"""
============================================================================
ЯЧЕЙКА 2: СТРУКТУРЫ ДАННЫХ
============================================================================
"""

@dataclass
class OCRResult:
    """
    Результат OCR обработки одного изображения.
    
    Thread-safety: ✅ Immutable после создания
    """
    model_name: str           # Название модели (HunyuanOCR, Qwen3-VL, и т.д.)
    folder_name: str          # Название папки/книги
    filename: str             # Имя файла (page_001.png)
    page_number: int          # Номер страницы
    text: str                 # Распознанный текст
    tokens: int               # Количество токенов
    processing_time: float    # Время обработки (секунды)
    text_length: int          # Длина текста (символов)
    timestamp: str            # ISO timestamp
    error: Optional[str] = None  # Текст ошибки (если есть)
    
    def __post_init__(self):
        """Автозаполнение полей"""
        if not self.timestamp:
            self.timestamp = datetime.now().isoformat()
        if self.text_length == 0:
            self.text_length = len(self.text)


@dataclass
class ModelStats:
    """
    Статистика работы модели.
    
    Thread-safety: ✅ Создаётся после завершения всех потоков
    """
    model_name: str
    total_images: int
    folders_processed: int
    total_time: float
    avg_time_per_image: float
    total_tokens: int
    total_chars: int
    success_rate: float
    errors: int
    throughput: float  # images per second


@dataclass
class FolderStats:
    """
    Статистика по одной папке/книге.
    
    Thread-safety: ✅ Создаётся после завершения всех потоков
    """
    folder_name: str
    total_images: int
    total_time: float
    avg_time_per_image: float
    total_chars: int
    avg_chars_per_image: float
    success_rate: float


# Глобальное хранилище результатов
ALL_RESULTS = {}      # {model_name: [OCRResult, ...]}
FOLDER_GROUPS = {}    # {folder_name: [image_paths, ...]}
IMAGE_PATHS = []      # Список всех путей к изображениям

print("✅ Ячейка 2: Структуры данных определены")
print("   - OCRResult: результат обработки одного изображения")
print("   - ModelStats: статистика модели")
print("   - FolderStats: статистика по папке")


✅ Ячейка 2: Структуры данных определены
   - OCRResult: результат обработки одного изображения
   - ModelStats: статистика модели
   - FolderStats: статистика по папке


In [4]:
"""
============================================================================
ЯЧЕЙКА 3: КОНФИГУРАЦИЯ МОДЕЛЕЙ
============================================================================
"""

MODELS_CONFIG = {
    "HunyuanOCR": {
        "enabled": True,
        "port": 8000,
        "model_path": "/models/HunyuanOCR",
        "prompts": {
            "simple": "OCR this text. Extract all text accurately.",
            "detailed": "Please extract all text from this image, preserving the original formatting, structure all formulas, symbols, equations and code blocs.",
            "math": "OCR this image containing mathematical text. Preserve all formulas, symbols, and equations."
        }
    },
    "Qwen3-VL-8B-Instruct": {
        "enabled": False,
        "port": 8001,
        "model_path": "/models/Qwen3-VL-8B-Instruct",
        "prompts": {
            "simple": "OCR this text.",
            "detailed": "Extract all text from this image.",
            "math": "OCR mathematical content from this image."
        }
    },
    "LightOnOCR-1B-1025": {
        "enabled": True,
        "port": 8000,
        "model_path": "/models/LightOnOCR-1B-1025",
        "prompts": {
            "simple": "OCR",
            "detailed": "Extract text",
            "math": "OCR math"
        }
    }
}

# Тип промпта для тестирования
PROMPT_TYPE = "simple"

print("✅ Ячейка 3: Конфигурация моделей загружена")
print(f"   Активных моделей: {sum(1 for m in MODELS_CONFIG.values() if m['enabled'])}")
print(f"   Тип промпта: {PROMPT_TYPE}")


✅ Ячейка 3: Конфигурация моделей загружена
   Активных моделей: 2
   Тип промпта: simple


In [5]:
"""
============================================================================
ЯЧЕЙКА 4: ЗАГРУЗКА ИЗОБРАЖЕНИЙ
============================================================================
"""

def load_images_from_directory(images_dir: Path) -> Tuple[List[Path], Dict[str, List[Path]]]:
    """
    Загружает изображения из директории, группируя по папкам.
    
    Returns:
        (image_paths, folder_groups)
    """
    image_paths = []
    folder_groups = {}
    
    # Поддерживаемые форматы
    image_extensions = {'.png', '.jpg', '.jpeg', '.webp', '.bmp'}
    
    # Рекурсивный обход
    for folder in sorted(images_dir.iterdir()):
        if not folder.is_dir():
            continue
        
        folder_name = folder.name
        folder_images = []
        
        for img_file in sorted(folder.iterdir()):
            if img_file.suffix.lower() in image_extensions:
                image_paths.append(img_file)
                folder_images.append(img_file)
        
        if folder_images:
            folder_groups[folder_name] = folder_images
            print(f"  📂 {folder_name}: {len(folder_images)} изображений")
    
    return image_paths, folder_groups


# Загрузка
print("🔍 Поиск изображений...")
IMAGE_PATHS, FOLDER_GROUPS = load_images_from_directory(IMAGES_DIR)

print(f"\n✅ Ячейка 4: Изображения загружены")
print(f"   Всего изображений: {len(IMAGE_PATHS)}")
print(f"   Папок: {len(FOLDER_GROUPS)}")


🔍 Поиск изображений...
  📂 o-predelnom-mnogomernom-raspredelenii: 4 изображений
  📂 standartizatsiya-i-kachestvo-zhizni: 3 изображений
  📂 Книга_Python_К_вершинам_мастерства_Рамальо_Лучано: 5 изображений

✅ Ячейка 4: Изображения загружены
   Всего изображений: 12
   Папок: 3


In [6]:
"""
============================================================================
ЯЧЕЙКА 5: ФУНКЦИИ СОХРАНЕНИЯ РЕЗУЛЬТАТОВ
============================================================================
"""

def save_results_to_files(
    results: List[OCRResult],
    output_dir: Path,
    model_name: str
) -> Dict[str, Path]:
    """
    Сохраняет результаты в различных форматах.
    
    Thread-safety: ✅ Вызывается после завершения всех потоков
    """
    saved_files = {}
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Группировка по папкам
    folder_groups = {}
    for result in results:
        if result.folder_name not in folder_groups:
            folder_groups[result.folder_name] = []
        folder_groups[result.folder_name].append(result)
    
    # 1. MARKDOWN файлы по папкам
    for folder_name, folder_results in folder_groups.items():
        md_path = output_dir / f"{model_name}_{folder_name}_{timestamp}.md"
        
        with open(md_path, 'w', encoding='utf-8') as f:
            f.write(f"# OCR Результаты: {folder_name}\n\n")
            f.write(f"Модель: {model_name}\n")
            f.write(f"Дата: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write(f"{'='*80}\n\n")
            
            for result in sorted(folder_results, key=lambda r: r.page_number):
                f.write(f"\n{'─'*80}\n")
                f.write(f"СТРАНИЦА {result.page_number}\n")
                f.write(f"Файл: {result.filename}\n")
                f.write(f"Время: {result.processing_time:.2f}с\n")
                f.write(f"Символов: {result.text_length}\n")
                
                if result.error:
                    f.write(f"❌ ОШИБКА: {result.error}\n")
                else:
                    f.write(f"{'─'*80}\n\n")
                    f.write(result.text)
                    f.write(f"\n\n")
        
        saved_files[f"md_{folder_name}"] = md_path
    
    # 2. JSON файл (полные результаты)
    json_path = output_dir / f"{model_name}_results_{timestamp}.json"
    json_data = [asdict(r) for r in results]
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=2)
    
    saved_files['json'] = json_path
    
    # 3. CSV файл (статистика)
    csv_path = output_dir / f"{model_name}_stats_{timestamp}.csv"
    
    with open(csv_path, 'w', encoding='utf-8', newline='') as f:
        fieldnames = [
            'folder_name', 'filename', 'page_number', 'text_length',
            'tokens', 'processing_time', 'error'
        ]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        
        for result in results:
            writer.writerow({
                'folder_name': result.folder_name,
                'filename': result.filename,
                'page_number': result.page_number,
                'text_length': result.text_length,
                'tokens': result.tokens,
                'processing_time': result.processing_time,
                'error': result.error or ''
            })
    
    saved_files['csv'] = csv_path
    
    return saved_files


print("✅ Ячейка 5: Функции сохранения определены")


✅ Ячейка 5: Функции сохранения определены


In [8]:
"""
============================================================================
ЯЧЕЙКА 6: ФУНКЦИИ СТАТИСТИКИ
============================================================================
"""

def calculate_model_stats(results: List[OCRResult]) -> ModelStats:
    """
    Вычисляет статистику для модели.
    
    Thread-safety: ✅ Вызывается после завершения всех потоков
    """
    if not results:
        return ModelStats(
            model_name="Unknown",
            total_images=0,
            folders_processed=0,
            total_time=0,
            avg_time_per_image=0,
            total_tokens=0,
            total_chars=0,
            success_rate=0,
            errors=0,
            throughput=0
        )
    
    model_name = results[0].model_name
    total_time = sum(r.processing_time for r in results)
    total_tokens = sum(r.tokens for r in results)
    total_chars = sum(r.text_length for r in results)
    
    # Успешные результаты (без ошибок)
    success_count = sum(1 for r in results if not r.error)
    success_rate = (success_count / len(results)) * 100 if results else 0
    
    # Количество ошибок
    errors = sum(1 for r in results if r.error)
    
    # Папки
    folders = set(r.folder_name for r in results)
    
    return ModelStats(
        model_name=model_name,
        total_images=len(results),
        folders_processed=len(folders),
        total_time=total_time,
        avg_time_per_image=total_time / len(results) if results else 0,
        total_tokens=total_tokens,
        total_chars=total_chars,
        success_rate=success_rate,
        errors=errors,
        throughput=len(results) / total_time if total_time > 0 else 0
    )


def calculate_folder_stats(results: List[OCRResult], folder_name: str) -> Optional[FolderStats]:
    """
    Вычисляет статистику для одной папки.
    
    Thread-safety: ✅ Вызывается после завершения потоков
    """
    folder_results = [r for r in results if r.folder_name == folder_name]
    
    if not folder_results:
        return None
    
    total_time = sum(r.processing_time for r in folder_results)
    total_chars = sum(r.text_length for r in folder_results)
    
    # Успешные результаты
    success_count = sum(1 for r in folder_results if not r.error)
    success_rate = (success_count / len(folder_results)) * 100 if folder_results else 0
    
    return FolderStats(
        folder_name=folder_name,
        total_images=len(folder_results),
        total_time=total_time,
        avg_time_per_image=total_time / len(folder_results) if folder_results else 0,
        total_chars=total_chars,
        avg_chars_per_image=total_chars / len(folder_results) if folder_results else 0,
        success_rate=success_rate
    )


print("✅ Ячейка 6: Функции статистики определены")


✅ Ячейка 6: Функции статистики определены


In [9]:
"""
============================================================================
ЯЧЕЙКА 7: ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
============================================================================
"""

def check_model_availability(port: int, timeout: int = 5) -> bool:
    """
    Проверяет доступность vLLM сервера.
    
    Thread-safety: ✅ Безопасна (только чтение)
    """
    try:
        response = requests.get(
            f"http://localhost:{port}/v1/models",
            timeout=timeout
        )
        return response.status_code == 200
    except Exception as e:
        logger.error(f"Порт {port} недоступен: {e}")
        return False


def image_to_base64(image: Image.Image, max_size: int = 1920) -> str:
    """
    Конвертирует PIL Image в base64 с автоматическим resize.
    
    Thread-safety: ✅ Безопасна (работает с копией изображения)
    """
    # Resize если изображение слишком большое
    if max(image.size) > max_size:
        ratio = max_size / max(image.size)
        new_size = (int(image.width * ratio), int(image.height * ratio))
        image = image.resize(new_size, Image.Resampling.LANCZOS)
    
    buffered = BytesIO()
    
    # Оптимизация формата
    if image.mode in ('RGBA', 'LA', 'P'):
        image.save(buffered, format="PNG", optimize=True)
    else:
        if image.mode != 'RGB':
            image = image.convert('RGB')
        image.save(buffered, format="JPEG", quality=95, optimize=True)
    
    return base64.b64encode(buffered.getvalue()).decode()


print("✅ Ячейка 7: Вспомогательные функции определены")


✅ Ячейка 7: Вспомогательные функции определены


In [10]:
"""
============================================================================
ЯЧЕЙКА 8: ПАРАЛЛЕЛЬНАЯ ОБРАБОТКА
============================================================================
"""

def ocr_single_image(
    img_path: Path,
    folder_name: str,
    model_name: str,
    model_config: Dict[str, Any],
    prompt_type: str,
    page_num: int
) -> OCRResult:
    """
    Обрабатывает одно изображение через OCR API.
    
    WORKER ФУНКЦИЯ для ThreadPoolExecutor
    
    Thread-safety: ✅ Полностью безопасна
    """
    start_time = time.time()
    
    try:
        # 1. Загрузка изображения
        image = Image.open(img_path)
        
        # 2. Конвертация в base64
        image_base64 = image_to_base64(image, max_size=1920)
        
        # 3. Создание API клиента
        client = OpenAI(
            base_url=f"http://localhost:{model_config['port']}/v1",
            api_key="EMPTY",
            timeout=180
        )
        
        # 4. Формирование запроса
        # ⚡ СПЕЦИАЛЬНАЯ ЛОГИКА ДЛЯ LightOnOCR
        if model_name == "LightOnOCR-1B-1025":
            # LightOnOCR: ТОЛЬКО изображение, БЕЗ текстового промпта
            messages = [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{image_base64}"
                            }
                        }
                    ]
                }
            ]
            
            # 5. Отправка запроса для LightOnOCR
            response = client.chat.completions.create(
                model=model_config["model_path"],
                messages=messages,
                temperature=0.2,
                top_p=0.9,
                max_tokens=model_config.get("safe_max_tokens", 4000)
            )
            
        else:
            # Другие модели: с текстовым промптом
            messages = [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": model_config["prompts"][prompt_type]
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{image_base64}"
                            }
                        }
                    ]
                }
            ]
            
            # 5. Отправка запроса для других моделей
            response = client.chat.completions.create(
                model=model_config["model_path"],
                messages=messages,
                temperature=0.0,
                max_tokens=model_config.get("safe_max_tokens", 4000)
            )
        
        
        # 6. Извлечение результата
        text = response.choices[0].message.content
        tokens = response.usage.total_tokens if response.usage else 0
        
        processing_time = time.time() - start_time
        
        # 7. Формирование результата
        return OCRResult(
            model_name=model_name,
            folder_name=folder_name,
            filename=img_path.name,
            page_number=page_num,
            text=text,
            tokens=tokens,
            processing_time=processing_time,
            text_length=len(text),
            timestamp=datetime.now().isoformat(),
            error=None
        )
        
    except Exception as e:
        processing_time = time.time() - start_time
        logger.error(f"Ошибка обработки {img_path.name}: {e}")
        
        return OCRResult(
            model_name=model_name,
            folder_name=folder_name,
            filename=img_path.name,
            page_number=page_num,
            text="",
            tokens=0,
            processing_time=processing_time,
            text_length=0,
            timestamp=datetime.now().isoformat(),
            error=str(e)
        )


def ocr_single_image_wrapper(args: Tuple) -> OCRResult:
    """Wrapper для распаковки аргументов"""
    return ocr_single_image(*args)


def test_model_parallel(
    model_name: str,
    max_workers: int = 4
) -> Optional[List[OCRResult]]:
    """
    ⚡⚡⚡ ПАРАЛЛЕЛЬНОЕ ТЕСТИРОВАНИЕ МОДЕЛИ ⚡⚡⚡
    """
    
    # Валидация
    if 'IMAGE_PATHS' not in globals() or not IMAGE_PATHS:
        print(f"❌ Изображения не загружены! Запустите Ячейку 4")
        return None
    
    if model_name not in MODELS_CONFIG:
        print(f"❌ Модель {model_name} не найдена в MODELS_CONFIG")
        return None
    
    model_config = MODELS_CONFIG[model_name]
    
    if not model_config["enabled"]:
        print(f"⏸️  {model_name} отключена в конфигурации")
        return None
    
    if not check_model_availability(model_config["port"]):
        print(f"❌ {model_name} недоступна на порту {model_config['port']}")
        return None
    
    # Заголовок
    print(f"{'='*70}")
    print(f"🚀 ПАРАЛЛЕЛЬНОЕ ТЕСТИРОВАНИЕ: {model_name}")
    print(f"{'='*70}")
    print(f"✅ Модель доступна на порту {model_config['port']}")
    print(f"📝 Промпт: {model_config['prompts'][PROMPT_TYPE]}")
    print(f"📊 Изображений: {len(IMAGE_PATHS)}")
    print(f"⚡ Параллельных потоков: {max_workers}")
    print(f"{'='*70}\n")
    
    # Подготовка задач
    tasks = []
    for folder_name, folder_paths in FOLDER_GROUPS.items():
        for page_num, img_path in enumerate(folder_paths, 1):
            task = (
                img_path,
                folder_name,
                model_name,
                model_config,
                PROMPT_TYPE,
                page_num
            )
            tasks.append(task)
    
    print(f"📦 Подготовлено задач: {len(tasks)}")
    print(f"   Worker будет обрабатывать: {len(tasks) / max_workers:.1f} задач в среднем")
    print(f"\n{'─'*70}\n")
    
    # Параллельная обработка
    start_time = time.time()
    results = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_task = {
            executor.submit(ocr_single_image_wrapper, task): task
            for task in tasks
        }
        
        with tqdm(total=len(tasks), desc=f"⚡ {model_name}", unit="img") as pbar:
            for future in as_completed(future_to_task):
                task = future_to_task[future]
                
                try:
                    result = future.result()
                    results.append(result)
                    
                    status = "✅" if not result.error else "❌"
                    pbar.set_postfix({
                        'файл': result.filename[:20],
                        'время': f"{result.processing_time:.1f}с",
                        'символов': result.text_length,
                        'статус': status
                    })
                    
                except Exception as exc:
                    logger.error(f"Worker exception: {exc}")
                    img_path = task[0]
                    results.append(OCRResult(
                        model_name=model_name,
                        folder_name=task[1],
                        filename=img_path.name,
                        page_number=task[5],
                        text="",
                        tokens=0,
                        processing_time=0,
                        text_length=0,
                        timestamp=datetime.now().isoformat(),
                        error=f"Worker exception: {exc}"
                    ))
                
                pbar.update(1)
    
    total_time = time.time() - start_time
    
    # Сохранение результатов
    ALL_RESULTS[model_name] = results
    
    print(f"\n💾 Сохранение результатов...")
    saved_files = save_results_to_files(results, OUTPUT_DIR, model_name)
    
    # Статистика
    stats = calculate_model_stats(results)
    
    # Вывод отчёта
    print(f"\n{'='*70}")
    print(f"📊 СТАТИСТИКА {model_name}")
    print(f"{'='*70}")
    print(f"  Изображений:      {stats.total_images}")
    print(f"  Папок:            {stats.folders_processed}")
    print(f"  Общее время:      {stats.total_time:.1f}с ({stats.total_time/60:.1f} мин)")
    print(f"  Throughput:       {stats.throughput:.2f} изобр/с")
    print(f"  Среднее время:    {stats.avg_time_per_image:.2f}с/изобр")
    
    # Оценка ускорения
    theoretical_speedup = max_workers / 1.5
    actual_speedup = (stats.avg_time_per_image * len(results)) / stats.total_time
    print(f"  Ускорение:        {actual_speedup:.1f}x (теор. {theoretical_speedup:.1f}x) ⚡")
    
    print(f"  Токенов:          {stats.total_tokens:,}")
    print(f"  Символов:         {stats.total_chars:,}")
    print(f"  Успешно:          {stats.success_rate:.1f}%")
    
    if stats.errors > 0:
        print(f"  ⚠️  Ошибок:       {stats.errors}")
    
    # Статистика по папкам
    if len(FOLDER_GROUPS) > 1:
        print(f"\n📂 По папкам:")
        for folder_name in FOLDER_GROUPS.keys():
            folder_stats = calculate_folder_stats(results, folder_name)
            if folder_stats:
                print(f"  {folder_name}: {folder_stats.total_images} изобр, "
                      f"{folder_stats.avg_time_per_image:.2f}с/изобр, "
                      f"{folder_stats.avg_chars_per_image:.0f} симв/изобр")
    
    # Сохранённые файлы
    print(f"\n💾 Файлы: {len(saved_files)}")
    for key, filepath in saved_files.items():
        print(f"  📄 {filepath.name}")
    
    print(f"\n{'='*70}")
    print(f"✅ ЗАВЕРШЕНО!")
    print(f"{'='*70}\n")
    
    return results


def test_model(model_name: str, max_workers: int = 4):
    """Алиас с автоматическим выбором workers"""
    if 'IMAGE_PATHS' in globals():
        num_images = len(IMAGE_PATHS)
        if num_images <= 4:
            max_workers = 2
        elif num_images <= 12:
            max_workers = 4
        else:
            max_workers = 8
    
    return test_model_parallel(model_name, max_workers=max_workers)


print("✅ Ячейка 8: Функции параллельной обработки определены")
print("   - ocr_single_image() - worker функция")
print("   - test_model_parallel() - главная функция")
print("   - test_model() - алиас с авто-выбором workers")


✅ Ячейка 8: Функции параллельной обработки определены
   - ocr_single_image() - worker функция
   - test_model_parallel() - главная функция
   - test_model() - алиас с авто-выбором workers


---

In [9]:
test_model_parallel("HunyuanOCR", max_workers=1)

🚀 ПАРАЛЛЕЛЬНОЕ ТЕСТИРОВАНИЕ: HunyuanOCR
✅ Модель доступна на порту 8000
📝 Промпт: OCR this text. Extract all text accurately.
📊 Изображений: 12
⚡ Параллельных потоков: 1

📦 Подготовлено задач: 12
   Worker будет обрабатывать: 12.0 задач в среднем

──────────────────────────────────────────────────────────────────────



⚡ HunyuanOCR:   0%|          | 0/12 [00:00<?, ?img/s]

2025-12-20 02:32:57,864 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ HunyuanOCR:   8%|▊         | 1/12 [02:21<25:56, 141.49s/img, файл=page_001.png, время=141.5с, символов=3294, статус=✅]

2025-12-20 02:33:37,966 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


...

⚡ HunyuanOCR:  92%|█████████▏| 11/12 [11:08<00:46, 46.70s/img, файл=page_004.png, время=61.0с, символов=1996, статус=✅]

2025-12-20 02:42:08,806 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ HunyuanOCR: 100%|██████████| 12/12 [11:32<00:00, 57.70s/img, файл=page_005.png, время=24.3с, символов=2487, статус=✅]


💾 Сохранение результатов...

📊 СТАТИСТИКА HunyuanOCR
  Изображений:      12
  Папок:            3
  Общее время:      692.4с (11.5 мин)
  Throughput:       0.02 изобр/с
  Среднее время:    57.70с/изобр
  Ускорение:        1.0x (теор. 0.7x) ⚡
  Токенов:          33,879
  Символов:         36,345
  Успешно:          100.0%

📂 По папкам:
  o-predelnom-mnogomernom-raspredelenii: 4 изобр, 65.01с/изобр, 3133 симв/изобр
  standartizatsiya-i-kachestvo-zhizni: 3 изобр, 83.68с/изобр, 4667 симв/изобр
  Книга_Python_К_вершинам_мастерства_Рамальо_Лучано: 5 изобр, 36.26с/изобр, 1962 симв/изобр

💾 Файлы: 5
  📄 HunyuanOCR_o-predelnom-mnogomernom-raspredelenii_20251220_024208.md
  📄 HunyuanOCR_standartizatsiya-i-kachestvo-zhizni_20251220_024208.md
  📄 HunyuanOCR_Книга_Python_К_вершинам_мастерства_Рамальо_Лучано_20251220_024208.md
  📄 HunyuanOCR_results_20251220_024208.json
  📄 HunyuanOCR_stats_20251220_024208.csv

✅ ЗАВЕРШЕНО!



[OCRResult(model_name='HunyuanOCR', folder_name='o-predelnom-mnogomernom-raspredelenii', filename='page_001.png', page_number=1, text="ВЕСТН. МОСК. УН-ТА. СЕР. 15. ВЫЧИСЛ. МАТЕМ. И КИБЕРН. 2017. № 3\n\n25\n\n---\n\n# СПИСОК ЛИТЕРАТУРЫ\n\n1. Бодров А.Г., Никитин А.А. Качественный и численный анализ интегрального уравнения, возникающего в модели стационарных сообществ // Докл. АН. 2014. **455**. № 5. С. 507–511.\n\n2. Бодров А.Г., Никитин А.А. Исследование интегрального уравнения плотности биологического вида в пространствах различных размерностей // Вестн. Моск. ун-та. Сер. 15. Вычисл. матем. и киберн. 2015. № 4. С. 7–13.\n\n3. Plank M.J., Law R. Spatial point processes and moment dynamics in the life sciences: a parsimonious derivation and some extensions // Bull. Math. Biol. 2015. **77**. P. 586–613.\n\n4. Dieckmann U., Law R. Relaxation projections and the method of moments // The Geometry of Ecological Interactions: Simplifying Spatial Complexity. Cambridge University Press, 2000. P

---

In [9]:
test_model_parallel("HunyuanOCR", max_workers=1)

🚀 ПАРАЛЛЕЛЬНОЕ ТЕСТИРОВАНИЕ: HunyuanOCR
✅ Модель доступна на порту 8000
📝 Промпт: OCR this text. Extract all text accurately.
📊 Изображений: 12
⚡ Параллельных потоков: 1

📦 Подготовлено задач: 12
   Worker будет обрабатывать: 12.0 задач в среднем

──────────────────────────────────────────────────────────────────────



⚡ HunyuanOCR:   0%|          | 0/12 [00:00<?, ?img/s]

2025-12-21 14:00:40,607 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ HunyuanOCR:   8%|▊         | 1/12 [02:54<31:57, 174.34s/img, файл=page_001.png, время=174.3с, символов=3294, статус=✅]

2025-12-21 14:01:26,161 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


...

⚡ HunyuanOCR:  92%|█████████▏| 11/12 [13:18<00:55, 55.60s/img, файл=page_004.png, время=74.9с, символов=1996, статус=✅]

2025-12-21 14:11:30,646 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ HunyuanOCR: 100%|██████████| 12/12 [13:44<00:00, 68.70s/img, файл=page_005.png, время=26.2с, символов=2489, статус=✅]


💾 Сохранение результатов...

📊 СТАТИСТИКА HunyuanOCR
  Изображений:      12
  Папок:            3
  Общее время:      824.4с (13.7 мин)
  Throughput:       0.01 изобр/с
  Среднее время:    68.70с/изобр
  Ускорение:        1.0x (теор. 0.7x) ⚡
  Токенов:          33,718
  Символов:         35,925
  Успешно:          100.0%

📂 По папкам:
  o-predelnom-mnogomernom-raspredelenii: 4 изобр, 78.25с/изобр, 3138 симв/изобр
  standartizatsiya-i-kachestvo-zhizni: 3 изобр, 99.05с/изобр, 4521 симв/изобр
  Книга_Python_К_вершинам_мастерства_Рамальо_Лучано: 5 изобр, 42.85с/изобр, 1962 симв/изобр

💾 Файлы: 5
  📄 HunyuanOCR_o-predelnom-mnogomernom-raspredelenii_20251221_141130.md
  📄 HunyuanOCR_standartizatsiya-i-kachestvo-zhizni_20251221_141130.md
  📄 HunyuanOCR_Книга_Python_К_вершинам_мастерства_Рамальо_Лучано_20251221_141130.md
  📄 HunyuanOCR_results_20251221_141130.json
  📄 HunyuanOCR_stats_20251221_141130.csv

✅ ЗАВЕРШЕНО!



[OCRResult(model_name='HunyuanOCR', folder_name='o-predelnom-mnogomernom-raspredelenii', filename='page_001.png', page_number=1, text="ВЕСТН. МОСК. УН-ТА. СЕР. 15. ВЫЧИСЛ. МАТЕМ. И КИБЕРН. 2017. № 3\n\n25\n\n---\n\n# СПИСОК ЛИТЕРАТУРЫ\n\n1. Бодров А.Г., Никитин А.А. Качественный и численный анализ интегрального уравнения, возникающего в модели стационарных сообществ // Докл. АН. 2014. **455**. № 5. С. 507–511.\n\n2. Бодров А.Г., Никитин А.А. Исследование интегрального уравнения плотности биологического вида в пространствах различных размерностей // Вестн. Моск. ун-та. Сер. 15. Вычисл. матем. и киберн. 2015. № 4. С. 7–13.\n\n3. Plank M.J., Law R. Spatial point processes and moment dynamics in the life sciences: a parsimonious derivation and some extensions // Bull. Math. Biol. 2015. **77**. P. 586–613.\n\n4. Dieckmann U., Law R. Relaxation projections and the method of moments // The Geometry of Ecological Interactions: Simplifying Spatial Complexity. Cambridge University Press, 2000. P

In [ ]:
test_model_parallel("HunyuanOCR", max_workers=1)

🚀 ПАРАЛЛЕЛЬНОЕ ТЕСТИРОВАНИЕ: HunyuanOCR
✅ Модель доступна на порту 8000
📝 Промпт: OCR this text. Extract all text accurately.
📊 Изображений: 12
⚡ Параллельных потоков: 1

📦 Подготовлено задач: 12
   Worker будет обрабатывать: 12.0 задач в среднем

──────────────────────────────────────────────────────────────────────



⚡ HunyuanOCR:   0%|          | 0/12 [00:00<?, ?img/s]

2025-12-21 14:43:10,088 - INFO - Retrying request to /chat/completions in 0.424031 seconds
2025-12-21 14:43:59,378 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ HunyuanOCR:   8%|▊         | 1/12 [03:49<42:03, 229.43s/img, файл=page_001.png, время=229.4с, символов=3294, статус=✅]

2025-12-21 14:44:48,615 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ HunyuanOCR:  17%|█▋        | 2/12 [04:38<20:34, 123.44s/img, файл=page_002.png, время=49.2с, символов=2977, статус=✅] 

2025-12-21 14:45:54,009 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ HunyuanOCR:  25%|██▌       | 3/12 [05:44<14:32, 96.93s/img, файл=page_003.png, время=65.4с, символов=4000, статус=✅] 

2025-12-21 14:46:29,348 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ HunyuanOCR:  33%|███▎      | 4/12 [06:19<09:40, 72.62s/img, файл=page_004.png, время=35.3с, символов=2295, статус=✅]

In [10]:
test_model_parallel("LightOnOCR-1B-1025", max_workers=1)

🚀 ПАРАЛЛЕЛЬНОЕ ТЕСТИРОВАНИЕ: LightOnOCR-1B-1025
✅ Модель доступна на порту 8000
📝 Промпт: OCR
📊 Изображений: 12
⚡ Параллельных потоков: 1

📦 Подготовлено задач: 12
   Worker будет обрабатывать: 12.0 задач в среднем

──────────────────────────────────────────────────────────────────────



⚡ LightOnOCR-1B-1025:   0%|          | 0/12 [00:00<?, ?img/s]

2025-12-21 17:50:52,799 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ LightOnOCR-1B-1025:   8%|▊         | 1/12 [01:14<13:41, 74.64s/img, файл=page_001.png, время=74.6с, символов=21, статус=✅]

2025-12-21 17:50:54,282 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


...

⚡ LightOnOCR-1B-1025:  92%|█████████▏| 11/12 [02:45<00:17, 17.76s/img, файл=page_004.png, время=40.1с, символов=21, статус=✅]

2025-12-21 17:52:24,050 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"


⚡ LightOnOCR-1B-1025: 100%|██████████| 12/12 [02:45<00:00, 13.82s/img, файл=page_005.png, время=0.8с, символов=21, статус=✅] 


💾 Сохранение результатов...

📊 СТАТИСТИКА LightOnOCR-1B-1025
  Изображений:      12
  Папок:            3
  Общее время:      165.9с (2.8 мин)
  Throughput:       0.07 изобр/с
  Среднее время:    13.82с/изобр
  Ускорение:        1.0x (теор. 0.7x) ⚡
  Токенов:          20,798
  Символов:         213
  Успешно:          100.0%

📂 По папкам:
  o-predelnom-mnogomernom-raspredelenii: 4 изобр, 19.67с/изобр, 11 симв/изобр
  standartizatsiya-i-kachestvo-zhizni: 3 изобр, 1.42с/изобр, 21 симв/изобр
  Книга_Python_К_вершинам_мастерства_Рамальо_Лучано: 5 изобр, 16.59с/изобр, 21 симв/изобр

💾 Файлы: 5
  📄 LightOnOCR-1B-1025_o-predelnom-mnogomernom-raspredelenii_20251221_175224.md
  📄 LightOnOCR-1B-1025_standartizatsiya-i-kachestvo-zhizni_20251221_175224.md
  📄 LightOnOCR-1B-1025_Книга_Python_К_вершинам_мастерства_Рамальо_Лучано_20251221_175224.md
  📄 LightOnOCR-1B-1025_results_20251221_175224.json
  📄 LightOnOCR-1B-1025_stats_20251221_175224.csv

✅ ЗАВЕРШЕНО!



[OCRResult(model_name='LightOnOCR-1B-1025', folder_name='o-predelnom-mnogomernom-raspredelenii', filename='page_001.png', page_number=1, text='![image](image_1.png)', tokens=2166, processing_time=74.64216709136963, text_length=21, timestamp='2025-12-21T17:50:52.803021', error=None),
 OCRResult(model_name='LightOnOCR-1B-1025', folder_name='o-predelnom-mnogomernom-raspredelenii', filename='page_002.png', page_number=2, text='![image](image_1.png)', tokens=2166, processing_time=1.4795749187469482, text_length=21, timestamp='2025-12-21T17:50:54.282830', error=None),
 OCRResult(model_name='LightOnOCR-1B-1025', folder_name='o-predelnom-mnogomernom-raspredelenii', filename='page_003.png', page_number=3, text='C1', tokens=2160, processing_time=1.275212287902832, text_length=2, timestamp='2025-12-21T17:50:55.558078', error=None),
 OCRResult(model_name='LightOnOCR-1B-1025', folder_name='o-predelnom-mnogomernom-raspredelenii', filename='page_004.png', page_number=4, text='C', tokens=2159, process

In [20]:
from openai import OpenAI
from PIL import Image
import base64
from io import BytesIO

def image_to_base64(image, max_size=1920):
    if max(image.size) > max_size:
        image.thumbnail((max_size, max_size), Image.Resampling.LANCZOS)
    buffer = BytesIO()
    image.save(buffer, format="PNG")
    return base64.b64encode(buffer.getvalue()).decode()

# 1. Загрузка тестового изображения
test_image_path = list(IMAGE_PATHS)[0]
print(f"📷 Тестовое изображение: {test_image_path.name}")

image = Image.open(test_image_path)
image_base64 = image_to_base64(image)

print(f"✅ Base64 длина: {len(image_base64):,} символов")
print(f"✅ Размер изображения: {image.size}")

# 2. API клиент
client = OpenAI(
    base_url="http://localhost:8000/v1",
    api_key="EMPTY",
    timeout=180
)

# 3. ТЕСТ 1: Без текстового промпта (правильный способ)
print("\n" + "="*70)
print("ТЕСТ 1: БЕЗ текстового промпта (правильный для LightOnOCR)")
print("="*70)

messages_correct = [
    {
        "role": "user",
        "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{image_base64}"
                }
            }
        ]
    }
]

try:
    response = client.chat.completions.create(
        model="/models/LightOnOCR-1B-1025",
        messages=messages_correct,
        temperature=0.2,
        top_p=0.9,
        max_tokens=4000
    )
    
    text = response.choices[0].message.content
    print(f"\n✅ РЕЗУЛЬТАТ:")
    print(f"   Символов: {len(text)}")
    print(f"   Токенов: {response.usage.total_tokens if response.usage else 'N/A'}")
    print(f"\n📝 ТЕКСТ (первые 500 символов):")
    print(text[:500])
    
except Exception as e:
    print(f"❌ ОШИБКА: {e}")

# 4. ТЕСТ 2: С текстовым промптом (неправильный способ)
print("\n" + "="*70)
print("ТЕСТ 2: С текстовым промптом (может не работать)")
print("="*70)

messages_wrong = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "Transcribe the text in this image."
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/png;base64,{image_base64}"
                }
            }
        ]
    }
]

try:
    response = client.chat.completions.create(
        model="/models/LightOnOCR-1B-1025",
        messages=messages_wrong,
        temperature=0.2,
        top_p=0.9,
        max_tokens=4000
    )
    
    text = response.choices[0].message.content
    print(f"\n✅ РЕЗУЛЬТАТ:")
    print(f"   Символов: {len(text)}")
    print(f"   Токенов: {response.usage.total_tokens if response.usage else 'N/A'}")
    print(f"\n📝 ТЕКСТ (первые 500 символов):")
    print(text[:500])
    
except Exception as e:
    print(f"❌ ОШИБКА: {e}")


📷 Тестовое изображение: page_001.png
✅ Base64 длина: 276,668 символов
✅ Размер изображения: (1211, 1713)

ТЕСТ 1: БЕЗ текстового промпта (правильный для LightOnOCR)
2025-12-21 19:03:24,723 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"

✅ РЕЗУЛЬТАТ:
   Символов: 21
   Токенов: 2166

📝 ТЕКСТ (первые 500 символов):
![image](image_1.png)

ТЕСТ 2: С текстовым промптом (может не работать)
2025-12-21 19:03:36,501 - INFO - HTTP Request: POST http://localhost:8000/v1/chat/completions "HTTP/1.1 200 OK"

✅ РЕЗУЛЬТАТ:
   Символов: 57
   Токенов: 2182

📝 ТЕКСТ (первые 500 символов):
Transcribe the text in this image.

![image](image_1.png)


In [11]:
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"/models/LightOnOCR-1B-1025","object":"model","created":1766350138,"owned_by":"vllm","root":"/models/LightOnOCR-1B-1025","parent":null,"max_model_len":4096,"permission":[{"id":"modelperm-af14aedc411310e6","object":"model_permission","created":1766350138,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [19]:
# Проверить что base64 не обрезан
image = Image.open(test_image_path)
buffer = BytesIO()
image.save(buffer, format="PNG")
image_bytes = buffer.getvalue()

print(f"Оригинальный размер: {len(image_bytes):,} байт")
print(f"Base64 размер: {len(image_base64):,} символов")
print(f"Ожидаемый Base64: {len(image_bytes) * 4 / 3:.0f} символов")


Оригинальный размер: 207,499 байт
Base64 размер: 276,668 символов
Ожидаемый Base64: 276665 символов


In [8]:
# Запустите в новой ячейке Jupyter
from transformers import AutoProcessor
import torch

model_path = PROJECT_ROOT / "models" / "LightOnOCR-1B-1025"

try:
    processor = AutoProcessor.from_pretrained(
        model_path,
        trust_remote_code=True
    )
    print("✅ Processor загружен успешно")
    print(f"📊 Тип: {type(processor)}")
    print(f"📊 Атрибуты: {dir(processor)}")
    
    # Проверить image_processor
    if hasattr(processor, 'image_processor'):
        print(f"✅ Image processor найден: {type(processor.image_processor)}")
    else:
        print(f"❌ Image processor ОТСУТСТВУЕТ!")
        
except Exception as e:
    print(f"❌ ОШИБКА загрузки processor: {e}")


✅ Processor загружен успешно
📊 Тип: <class 'transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast'>
📊 Атрибуты: ['SPECIAL_TOKENS_ATTRIBUTES', '__annotations__', '__bool__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_tokens', '_auto_class', '_batch_encode_plus', '_call_one', '_convert_encoding', '_convert_id_to_token', '_convert_token_to_id_with_added_voc', '_create_repo', '_decode', '_decode_use_source_tokenizer', '_encode_plus', '_eventual_warn_about_too_long_sequence', '_eventually_correct_t5_max_length', '_from_pretrained', '_get_files_timestamps', '_get_padding_truncation_strategies', '_in_target_context_manager', '_pad', '_pad_

In [10]:
# Проверка что модель вообще работает локально
from transformers import AutoModelForVision2Seq, AutoProcessor
from PIL import Image
import torch

model_path = PROJECT_ROOT / "models" / "LightOnOCR-1B-1025"

print("🔄 Загрузка модели напрямую (без vLLM)...")

try:
    # Загрузка
    processor = AutoProcessor.from_pretrained(
        model_path,
        trust_remote_code=True
    )
    
    model = AutoModelForVision2Seq.from_pretrained(
        model_path,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        device_map="cuda:0"  # или "cpu" если нет GPU
    )
    
    print("✅ Модель загружена")
    
    # Тестовое изображение
    test_image = list(IMAGE_PATHS)[0]
    image = Image.open(test_image)
    
    # Обработка
    inputs = processor(images=image, return_tensors="pt").to(model.device)
    
    # Генерация
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.2,
            top_p=0.9
        )
    
    # Декодирование
    text = processor.batch_decode(outputs, skip_special_tokens=True)[0]
    
    print(f"\n📝 РЕЗУЛЬТАТ:")
    print(f"   Символов: {len(text)}")
    print(f"\n{text[:500]}")
    
except Exception as e:
    print(f"❌ ОШИБКА: {e}")
    import traceback
    traceback.print_exc()


🔄 Загрузка модели напрямую (без vLLM)...
❌ ОШИБКА: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires `accelerate`. You can install it with `pip install accelerate`


Traceback (most recent call last):
  File "/tmp/ipykernel_617726/2282465356.py", line 17, in <module>
    model = AutoModelForVision2Seq.from_pretrained(
  File "/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py", line 2289, in from_pretrained
    return super().from_pretrained(pretrained_model_name_or_path, *model_args, **kwargs)
  File "/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py", line 604, in from_pretrained
    return model_class.from_pretrained(
  File "/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/transformers/modeling_utils.py", line 277, in _wrapper
    return func(*args, **kwargs)
  File "/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/transformers/modeling_utils.py", line 4806, in from_pretrained
    raise ValueError(
ValueError: Using a `device_map`, `tp_plan`, `torch.device` context manager or setting `torch.set_default_device(device)` requires

In [13]:
"""
Правильная загрузка LightOnOCR с кастомным процессором
"""

from transformers import AutoModel, AutoTokenizer, AutoImageProcessor
from PIL import Image
import torch
import sys

model_path = PROJECT_ROOT / "models" / "LightOnOCR-1B-1025"

print("="*70)
print("🔧 ПРАВИЛЬНАЯ ЗАГРУЗКА LightOnOCR")
print("="*70)

try:
    # Шаг 1: Загрузить модель с trust_remote_code
    print(f"\n📦 Загрузка модели...")
    model = AutoModel.from_pretrained(
        model_path,
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        device_map="auto"
    )
    print(f"✅ Модель: {type(model)}")
    
    # Шаг 2: Загрузить токенизатор
    print(f"\n📦 Загрузка токенизатора...")
    tokenizer = AutoTokenizer.from_pretrained(
        model_path,
        trust_remote_code=True
    )
    print(f"✅ Токенизатор: {type(tokenizer)}")
    
    # Шаг 3: Загрузить image processor
    print(f"\n📦 Загрузка image processor...")
    try:
        image_processor = AutoImageProcessor.from_pretrained(
            model_path,
            trust_remote_code=True
        )
        print(f"✅ Image Processor: {type(image_processor)}")
    except Exception as e:
        print(f"⚠️  AutoImageProcessor не сработал: {e}")
        print(f"   Попытка загрузить из preprocessor_config.json...")
        
        # Попробовать загрузить вручную
        from transformers import PreTrainedImageProcessor
        import json
        
        with open(f"{model_path}/preprocessor_config.json", "r") as f:
            config = json.load(f)
        
        print(f"   Preprocessor config: {config}")
    
    # Шаг 4: ТЕСТ - Обработка изображения
    print(f"\n🧪 ТЕСТ: Обработка изображения...")
    
    test_image = list(IMAGE_PATHS)[0]
    image = Image.open(test_image)
    print(f"   Изображение: {test_image.name} ({image.size})")
    
    # Попробовать разные способы обработки
    print(f"\n   Способ 1: Через модель напрямую...")
    if hasattr(model, 'forward'):
        # Подготовить inputs
        # LightOnOCR ожидает pixel_values
        from torchvision import transforms
        
        # Простая нормализация
        transform = transforms.Compose([
            transforms.Resize((512, 512)),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.48145466, 0.4578275, 0.40821073],
                std=[0.26862954, 0.26130258, 0.27577711]
            )
        ])
        
        pixel_values = transform(image).unsqueeze(0).to(model.device)
        print(f"   ✅ pixel_values shape: {pixel_values.shape}")
        
        # Генерация
        with torch.no_grad():
            outputs = model.generate(
                pixel_values=pixel_values,
                max_new_tokens=512,
                temperature=0.2,
                top_p=0.9,
                do_sample=True
            )
        
        # Декодирование
        text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        print(f"\n📝 РЕЗУЛЬТАТ:")
        print(f"   Символов: {len(text)}")
        print(f"\n{text[:500]}")
        
        if len(text) > 50:
            print(f"\n🎉 УСПЕХ! Модель распознала текст!")
        else:
            print(f"\n⚠️  Результат слишком короткий")
    
except Exception as e:
    print(f"\n❌ ОШИБКА: {e}")
    import traceback
    traceback.print_exc()

print(f"\n{'='*70}")


`torch_dtype` is deprecated! Use `dtype` instead!


🔧 ПРАВИЛЬНАЯ ЗАГРУЗКА LightOnOCR

📦 Загрузка модели...
2025-12-21 23:49:34,712 - INFO - We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Some weights of Mistral3Model were not initialized from the model checkpoint at /home/gna/workspase/education/MEPHI/Team_5_AI_Tutor/models/LightOnOCR-1B-1025 and are newly initialized: ['language_model.embed_tokens.weight', 'language_model.layers.0.input_layernorm.weight', 'language_model.layers.0.mlp.down_proj.weight', 'language_model.layers.0.mlp.gate_proj.weight', 'language_model.layers.0.mlp.up_proj.weight', 'language_model.layers.0.post_attention_layernorm.weight', 'language_model.layers.0.self_attn.k_norm.weight', 'language_model.layers.0.self_attn.k_proj.weight', 'language_model.layers.0.self_attn.o_proj.weight', 'language_model.layers.0.self_attn.q_norm.weight', 'language_model.layers.0.self_attn.q_proj.weight', 'language_model.layers.0.self_attn.v_proj.weight', 'language_model.layers.1.input_layernorm.weight', 'language_model.layers.1.mlp.down_proj.weight', 'language_model.layers.1.mlp.gate_proj.weight', 'language_model.layers.1.mlp.up_proj.weight', 'language_model.layers.1.po

✅ Модель: <class 'transformers.models.mistral3.modeling_mistral3.Mistral3Model'>

📦 Загрузка токенизатора...
✅ Токенизатор: <class 'transformers.models.qwen2.tokenization_qwen2_fast.Qwen2TokenizerFast'>

📦 Загрузка image processor...
✅ Image Processor: <class 'transformers.models.pixtral.image_processing_pixtral_fast.PixtralImageProcessorFast'>

🧪 ТЕСТ: Обработка изображения...
   Изображение: page_001.png ((1211, 1713))

   Способ 1: Через модель напрямую...
   ✅ pixel_values shape: torch.Size([1, 3, 512, 512])

❌ ОШИБКА: 'Mistral3Model' object has no attribute 'generate'



Traceback (most recent call last):
  File "/tmp/ipykernel_1025398/1642129295.py", line 85, in <module>
    outputs = model.generate(
  File "/home/gna/anaconda3/envs/rocm/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1931, in __getattr__
    raise AttributeError(
AttributeError: 'Mistral3Model' object has no attribute 'generate'
